In [1]:
from dotenv import load_dotenv

load_dotenv()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

In [2]:
from openai import OpenAI

client = OpenAI()
messages = []
messages.append({"role": "system", "content": "You are a helpful assistant."})

def send_message(message: str):
    messages.append({"role": "user", "content": message})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message

for i in range(1,3):
    answer = send_message(input(f"{i} - 질문을 해주세요:"))
    print(f"{answer.content}\n----------------------------\n")

    messages.append({"role": "assistant", "content": answer.content})


선릉(宣陵)은 대한민국 서울특별시 강남구에 위치한 조선 왕조의 왕릉입니다. 선릉은 조선 제9대 왕인 성종(成宗)과 그의 왕비인 정희왕후(定熙王后)의 능으로, 성종은 1469년에 즉위하여 1494년에 사망하였습니다. 정희왕후는 성종의 재위 기간 동안 왕비로 있었으며, 성종의 사망 후에 그와 함께 모셔졌습니다.

선릉은 역사적으로 중요한 유적지로, 조선 왕릉 중 하나로 문화재 지정도 되어 있습니다. 이곳은 자연 환경과의 조화가 잘 이루어져 있으며, 전통적인 한국의 무덤 양식인 '원형' 무덤 형태로 조성되어 있습니다. 선릉은 왕릉의 여유로운 경관과 고요한 분위기로 인해 많은 방문객들이 찾고 있습니다.

또한, 선릉은 역사적인 의미 외에도 주변 공원과 함께 시민들에게 휴식 공간으로 제공되고 있어, 산책이나 운동을 즐기는 사람들에게 인기 있는 장소입니다. 이곳은 또한 UNESCO 세계유산으로 등재된 조선 왕릉의 일부분으로, 조선 왕조의 역사와 문화를 증명하는 중요한 유적입니다.
----------------------------



sent 3.09KB in 1.27181s to https://llmobs-intake.datadoghq.com/api/v2/llmobs


선릉에 묻힌 정희왕후(定熙王后)는 조선 제9대 왕 성종의 왕비로, 본명은 한씨이며, 성종과는 혼인하여 고종과 정비인 경빈이치(景嬪李氏), 그리고 후에 왕세자로 책봉된 관종(光宗) 등 자녀를 두었습니다. 

정희왕후는 성종과의 혼인으로 인해 왕비의 위치에 올랐으며, 왕비로서의 역할을 수행하는 동안 성종의 통치에 큰 영향을 미쳤다고 전해집니다. 그녀는 성종의 정치적, 경제적, 문화적 정책을 지지하고 조정하면서 조선의 중흥기에 기여했습니다. 

정희왕후는 학문을 중시하고 문화 발전에 관심을 가졌으며, 후대로 이어지는 왕비들의 전통과 역할 모델이 되었습니다. 성종이 사망한 후, 그녀는 그의 곁에 묻히기 위해 선릉에 위치하게 되었고, 그녀의 능은 성종의 능과 함께 조화롭게 조성되었습니다.

정희왕후는 그 자체로도 중요한 역사적 인물이지만, 조선 사회에서의 여성의 위치와 역할, 그리고 왕비의 정치적 영향력과 관련하여 현대의 연구에서도 흥미로운 주제로 다루어지고 있습니다.
----------------------------



In [7]:
from langchain_community.chat_models import ChatOpenAI, ChatOllama
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI( temperature=1.0, model="gpt-4o-mini" )
#llm = ChatOllama( temperature=1.0, model="llama3:latest" )

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant DD-BOT."),
    ("ai","Hi. My name is DD-BOT"),
    ("human", "{question}\n그리고 너를 뭐라고 부르면 될까?")
])

prompt = template.format_messages("선릉과 정릉에 대해 설명해줘.")
answer = llm.invoke(prompt)
print(f"{answer.content}\n----------------------------\n")

AIMessage(content='안녕하세요! 저를 DD-BOT이라고 부르셔도 좋고, 그냥 "봇"이라고 부르셔도 됩니다. 편하게 말씀해 주세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 45, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-f0f444a1-f766-423c-963b-2edfb17ef7d5-0')

In [6]:

# 위의 llm.invoke 로 바로 불러도 되지만, 아래처럼 chain 으로 부르면 더 편해짐
chain = template | llm
answer = chain.invoke({"question": input(f"질문을 해주세요:")})
print(f"{answer.content}\n----------------------------\n")


AIMessage(content='안녕하세요! 저를 "DDBOT"이라고 부르셔도 좋고, 그냥 "AI"나 "도움이 필요한 AI"라고 부르셔도 괜찮습니다. 여러분에게 도움이 필요한 질문이나 요청이 있다면 언제든지 말씀해 주세요!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 42, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-b57637da-753c-4497-aa10-0ce0b7983316-0')

In [16]:
# 결과를 정리하는 Parser 를 만들어서, 랭체인에 추가할 수 있음.
# DD 에서 task annotation 으로 추가도 가능.

from langchain.schema import BaseOutputParser
from ddtrace.llmobs.decorators import task

class CommaOutputParser(BaseOutputParser):
    @task()
    def parse(self, text: str):
        items = text.split(",")
        return list(map(str.strip, items))
    
template = ChatPromptTemplate.from_messages([
    ("system", "너는 List를 만드는 기계야. 앞으로 답변은 모두 comma로 구분짓고, 최대 {max_items} 개까지만 답변해. 다른 것은 답변하지마."),
    ("human", "{question}")
])

chain = template | llm | CommaOutputParser() 

chain.invoke({
    "max_items":5,
    "question":"조선시대 왕들을 읇어봐"
})

I/O object is not JSON serializable. Defaulting to placeholder value instead.


['태조', '정종', '태종', '세종', '문종']

In [22]:
# 랭체인으로 LLM Call 의 결과도 다시 사용가능

first_template = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant."),
  ("human", "{question}")
])

first_chain = first_template | llm
question = "선릉에 대해 설명해줘."
#result = first_chain.invoke({"question":question})
#result

pirate_prompt = ChatPromptTemplate.from_messages([
    ("system","너는 이제 조선시대 노비야, 이제 아래 들어오는 human 메세지를 조선시대 노비처럼 말해줘."),
    ("human","{human_message}")
])

pirate_chain = pirate_prompt | llm

final_chain = {"human_message":first_chain} | pirate_chain

result = final_chain.invoke({"question":question})
result.content

"하옵니다, 주인님. 선릉은 조선의 성종 대왕과 왕비 윤씨의 능이니, 역사적으로 매우 귀한 장소로 여겨지옵니다. 성종 대왕은 1469년부터 1494년까지 나라를 다스리셨고, 그 고운 통치 아래서 조선은 문화와 학문, 정치를 크게 발전시켰다 하옵니다.\n\n선릉은 서울 강남구에 위치하며, '원주형'이라 일컫는 한국의 전통적인 무덤 형태로 지어졌사옵니다. 여름과 겨울의 기후를 고려하여 정교하게 설계되었고, 왕과 왕비가 나란히 모셔져 있사옵니다. 주변은 자연과 조화를 이루도록 아름답게 꾸며져 있으며, 조선시대의 건축양식과 예술이 잘 드러나 있사옵니다.\n\n또한 선릉은 조선의 역사와 문화를 체험할 수 있는 중요한 장소로 여겨지며, 방문객들에게 많은 감동을 주는 곳이옵니다. 능역 안에는 다양한 조각과 비석, 정원이 조성되어 있어 문화재로서의 가치가 높아, 많은 관광객들이 이곳을 찾는답니다. 주인님도 한 번 다녀가시는 것이 어떠신지요?"

sent 23.05KB in 1.17043s to https://llmobs-intake.datadoghq.com/api/v2/llmobs


In [8]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS, Chroma

loader = TextLoader("./kings.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder( 
    separator="\n\n",
    chunk_size=1500, 
    chunk_overlap=100
)
docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()

# 문서 임베딩해서 벡터디비로 반환
vector_store = FAISS.from_documents(docs, embedder)


Created a chunk of size 2062, which is longer than the specified 1500
Created a chunk of size 4177, which is longer than the specified 1500
Created a chunk of size 1718, which is longer than the specified 1500
Created a chunk of size 4045, which is longer than the specified 1500
Created a chunk of size 2177, which is longer than the specified 1500
Created a chunk of size 1700, which is longer than the specified 1500
/var/folders/l4/gcv5br7s0pd3zwwmnyt30w9r0000gp/T/ipykernel_8821/1723271310.py:15: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedder = OpenAIEmbeddings()


[(Document(metadata={'source': './kings.txt'}, page_content='연산군\n1. 성종과 연산군이 오랜만에 밖에 바람 쐬러 나와서 성종이\n‘융아, 좋지 않으냐?’ 하니 연산군이 소떼를 보며 ‘아바마마 저 송아지도 어미 소가 있는데 저는 왜 어미가 없습니까’ 라고 말했다.\n2. 또 다른 야사에서는 연산이 동궁이었을 때 어느 날 성종에게 거리에서 나가 놀고 싶다고 하여 성종이 허락하였다. 저녁에 동궁이 궁궐로 돌아오자 성종이\n“오늘은 밖에서 무엇을 보았느냐?” 하고 묻자 연산은\n“구경할만한 것을 없었습니다. 다만 송아지 한 마리가 어미 소를 따라가는데 그 어미 소가 울면 송아지도 따라 우니 그것이 가장 부러운 일이었습니다.”\n하여 성종이 가여워 눈물을 흘렸다는 일화가 전해짐.\n3. 공부하는 걸 엄청 싫어하여, 왕자를 훈육하는 학자들이 엄청 고생을 했다고 한다.\n매일 꾸중 듣고 야단맞아, 폐비 윤씨 사건이 일어났을 때 모두 죽여버렸다.\n4. 춤을 잘 췄다고 한다.\n5. 모후인 폐비 윤씨와 고려 공민왕의 비인 노국대장공주의 얼굴이 비슷하게 생겼다는 이유로 관공서에 노국대장공주의 초상화를 수집하게 한 일도 있었다.\n6. 엄청난 꽃미남이었다고 한다.\n보너스. 연산군에 대한 잘못된 사실들\n연산군이 정치를 잘 하지 못하고 폭군의 모습을 다소 보인점은 사실이지만\n몇가지 왜곡된 사실들이 있다.\n1.연산군은 궁궐 담장 아래 백척 내에 있는 민가들을 철거하여 쫓아냈다\n-하지만 이것은 원래 불법주택들이였다. 선왕, 후궁들의 모습이 민간에 노출되어서는 안된다는\n법도에 의한 것들이였다. 그러나 연산군은 오히려 봄까지 기다려주고 주택의 등급을 나눠 무명을\n보상으로 주고 대토 와 거주지까지 마련해 주었다\n2.궁을 짓기 위해 열한 고을의 백성들을 내쫓았다\n- 백성들을 내쫓은것은 사실이지만 그 수는 500여호에 불과했고 열한고을은 사관이\n의도적으로 과장해서 쓴것이다. 또 지나가는 자는 죽여서 시신을 구경시키는 형을\n

sent 79.8KB in 1.20821s to https://llmobs-intake.datadoghq.com/api/v2/llmobs


In [ ]:

# 질문 임베딩 후에, 유사 문서 검색
query = "연산군과 관련된 것을 알려줘."
results = vector_store.similarity_search_with_relevance_scores(query)
results


In [9]:
# 쿼리와 유사한 문서들을 가져오고, 문서들을 토대로 답변하기
from langchain.schema.runnable import RunnablePassthrough

retriever = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 잘 답변하는 어시스턴트야. 이 뒤에 있는 입력된 문맥을 이용해서, 질문에 답변해줘. 답을 모른다면, 그냥 모른다고 답해. 억지롤 답을 만들면 안돼.\n\n{context}"),
    ("human", "{question}")
])

chain = (
    {
        "context":retriever, 
        "question":RunnablePassthrough()
    } 
    | prompt 
    | llm 
)

chain.invoke("어린 나이에 아버지를 잃은 왕을 일려줘.")


AIMessage(content='어린 나이에 아버지를 잃은 왕은 헌종입니다.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 1942, 'total_tokens': 1959, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reason': 'stop', 'logprobs': None}, id='run-58a4142c-021b-4e7a-8a85-83a1c43071fd-0')

sent 42.53KB in 1.05813s to https://llmobs-intake.datadoghq.com/api/v2/llmobs


In [21]:
# 쿼리와 유사한 문서들을 가져오고, 문서들을 Re-Ranking 해서 답변하기
from langchain.chains import RetrievalQA
from ddtrace.llmobs.decorators import workflow

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vector_store.as_retriever() 
)

@workflow
def answer_question(query):
    LLMObs.annotate(
        input_data=query,
    )
    result = chain.invoke(query)
    return result['result']

query = "조선 왕들 중 가장 많은 자식을 가진 왕에 대해 설명해줘."
#query = "조선 왕들 중 자식 사랑이 남달랐던 왕에 대해 설명해줘."
answer_question(query)


/Users/kihyun.lee/work/gpt-notebook-test/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'태종은 조선 왕들 중 가장 많은 자식을 가진 왕으로 12남 17녀, 총 29명의 자식을 두었다. 그는 자신의 자녀들이 정치와 사회에서 잘 성장할 수 있도록 많은 노력을 기울였던 것으로 보인다. 특히 태종은 현실론자였고, 그의 자녀들은 그가 남긴 유산과 시대의 변화 속에서 중요한 역할을 하게 되었다. 태종은 또한 아들 양녕대군의 행동에 속상해 밤새 울었던 일화가 있으며, 양녕대군의 아들 이혜는 더 많은 사고를 쳐 결국 왕자의 작위를 잃고 비극적인 결말을 맞았다. 이런 가족 내사적인 사건들은 태종의 복잡한 가정사와 자식 양육에 대한 그의 큰 영향을 보여준다. '

sent 124.72KB in 1.50415s to https://llmobs-intake.datadoghq.com/api/v2/llmobs
